<a href="https://colab.research.google.com/github/sainik-khaddar/SERB-INAC-FINALS/blob/main/TorchTextSummarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install transformers==2.8.0
 !pip install torch==1.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 563 kB 5.4 MB/s 
     |████████████████████████████████| 880 kB 40.9 MB/s 
     |████████████████████████████████| 5.6 MB 36.2 MB/s 
     |████████████████████████████████| 1.3 MB 44.9 MB/s 
     |████████████████████████████████| 132 kB 49.4 MB/s 
     |████████████████████████████████| 9.1 MB 30.6 MB/s 
     |████████████████████████████████| 79 kB 5.9 MB/s 
     |████████████████████████████████| 140 kB 57.5 MB/s 
     |████████████████████████████████| 127 kB 58.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=859f5683f6e582aa198969dd6c04788a4b27b270387a2847430fa0242279f4ed
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    U

In [2]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [3]:
# initialize the pretrained model
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [ ]:
# input text
text = """
The cardiovascular and respiratory autonomic nervous regulation has been studied mainly by hemodynamic responses during different physical stressors . In this study dynamics of autonomic response to an orthostatic challenge was investigated by hemodynamic variables and by diverse linear and nonlinear indices calculated from time series of beat to beat intervals respiratory cycle duration systolic and diastolic blood pressure . This study included 16 young female patients with vasovagal syncope and 12 age matched female controls . The subjects were enrolled in a head up tilt test breathing normally including 5min of baseline and 18min of 70 orthostatic phase . To increase the time resolution of the analysis the time series were segmented in five minute overlapping windows with a shift of 1min . Hemodynamic parameters did not show any statistical differences between SYN and CON . Time domain linear analysis revealed increased respiratory frequency and increased blood pressure variability in patients during OP meaning increased sympathetic activity and vagal withdrawal . Frequency domain analysis confirmed a predominance of sympathetic tone by steadily increased values of low over high frequency power in BBI and of low frequency power in SYS and DIA in patients during OP . The nonlinear analysis by symbolic dynamics seemed to be highly suitable for differentiation of SYN and CON in the early beginning of OP i.e . 5min after tilt up . In particular the index SYS plvar3 showed less patterns of low variability in patients reflecting a steadily increase in both BPV and sympathetic activity . The proposed dynamical analysis could lead to a better understanding of the temporal underlying mechanisms in healthy subjects and patients under orthostatic stress .
"""

In [ ]:
## preprocess the input text
preprocessed_text = text.strip().replace('\n','')
t5_input_text = 'summarize: ' + preprocessed_text

In [ ]:
t5_input_text

'summarize: The cardiovascular and respiratory autonomic nervous regulation has been studied mainly by hemodynamic responses during different physical stressors . In this study dynamics of autonomic response to an orthostatic challenge was investigated by hemodynamic variables and by diverse linear and nonlinear indices calculated from time series of beat to beat intervals respiratory cycle duration systolic and diastolic blood pressure . This study included 16 young female patients with vasovagal syncope and 12 age matched female controls . The subjects were enrolled in a head up tilt test breathing normally including 5min of baseline and 18min of 70 orthostatic phase . To increase the time resolution of the analysis the time series were segmented in five minute overlapping windows with a shift of 1min . Hemodynamic parameters did not show any statistical differences between SYN and CON . Time domain linear analysis revealed increased respiratory frequency and increased blood pressure

In [ ]:
len(t5_input_text.split())

277

In [ ]:
tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)

In [ ]:
summary_ids = model.generate(tokenized_text, min_length=30, max_length=100)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
summary

'the cardiovascular and respiratory autonomic nervous regulation has been studied mainly by hemodynamic responses during different physical stressors. in this study dynamics of autonomic response to an orthostatic challenge was investigated by hemodynamic variables and by diverse linear and nonlinear indices calculated from time series of beat to beat intervals respiratory cycle duration systolic and diastolic blood pressure.'

In [4]:
import pandas as pd
import numpy as np

In [5]:
df=pd.read_csv('train.csv')

In [6]:
for i in range(0,2000):
  df["Abstract"].iloc[i]=str(df["Abstract"].iloc[i])

In [7]:
df["Abstract"].iloc[1999]

' Background and objective This article presents a multimodal analysis of startle type responses using a variety of physiological facial and speech features . These multimodal components of the startle type response reflect complex brain body reactions to a sudden and intense stimulus . Additionally the proposed multimodal evaluation of reflexive and emotional reactions associated with the startle eliciting stimuli and underlying neural networks and pathways could be applied in diagnostics of different psychiatric and neurological diseases . Different startle type stimuli can be compared in the strength of their elicitation of startle responses i.e . their potential to activate stress related neural pathways underlying biomarkers and corresponding behavioral reactions . Methods An innovative method for measuring startle type responses using multimodal stimuli and multimodal feature analysis has been introduced . Individual s multimodal reflexive and emotional expressions during startle

In [8]:
df1 = pd.DataFrame(columns = ['RHS'])

In [ ]:
for i in range(0,2000):
  text=df["Abstract"].iloc[i]
  preprocessed_text = text.strip().replace('\n','')
  t5_input_text = 'summarize: ' + preprocessed_text
  tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)
  summary_ids = model.generate(tokenized_text, min_length=30, max_length=100)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  # print(str(i) + '. ' +summary)
  df1.loc[i] = [summary]
  print(df1)
  i=i+1

In [17]:
df1.to_csv('summary2.csv')